In [1]:
import pandas as pd
import arcpy
import os

In [2]:
# set workspace to your geodatabase
gdb = r"F:\GIS\DB_CONNECT\Vector.sde"
# Input feature classes


arcpy.env.overwriteOutput = True

bus_stop_fc = os.path.join(gdb, r"Transportation\Tahoe_Bus_Stops")
bus_stop_relationship = os.path.join(gdb, r"Transportation\Tahoe_Bus_Stops_Relationship")
bus_route_fc = os.path.join(gdb, r"Transportation\Tahoe_Transit_Routes_Consolidated")

# read these into pandas dataframes


In [3]:

def to_dataframe(fc):
    fields = [f.name for f in arcpy.ListFields(fc)]
    with arcpy.da.SearchCursor(fc, fields) as cursor:
        rows = [list(r) for r in cursor]
    return pd.DataFrame(rows, columns=fields)

# Read into DataFrames
df_stops = to_dataframe(bus_stop_fc)
df_stop_rel = to_dataframe(bus_stop_relationship)
df_routes = to_dataframe(bus_route_fc)

In [4]:
df_routes_clean = df_routes.dropna(subset=['Frequency']).reset_index(drop=True)


In [5]:
df_stop_rel_clean = df_stop_rel[['STOP_ID', 'ROUTE_NAME']].drop_duplicates().reset_index(drop=True)

In [6]:
df_stops_frequency = (
    df_stop_rel_clean
        .merge(df_routes_clean, on='ROUTE_NAME', how='left')
        .groupby('STOP_ID')
        .agg(
            MIN_FREQUENCY_MINUTES=('Frequency', 'min'),
            AVG_FREQUENCY_MINUTES=('Frequency', 'mean'),
            ROUTE_COUNT=('ROUTE_NAME', 'count')
        )
        .reset_index()
)


In [7]:
df_stops_frequency.to_csv(r"stop_quantity_frequency.csv", index=False)

In [ ]:
arcpy.stats.CalculateCompositeIndex(
    in_table="Tesselation_Clean",
    in_variables="INDEX_5 false;INDEX_2 false;Transit_Index #",
    append_to_input="APPEND_TO_INPUT",
    out_table=None,
    index_preset="MEAN_SCALED",
    preprocessing="MINMAX",
    pre_threshold_scaling="THRESHOLD_PERCENTILE",
    pre_custom_zscore=None,
    pre_min_max=None,
    pre_thresholds=None,
    index_method="MEAN",
    index_weights="INDEX_5 1;INDEX_2 1;Transit_Index 1",
    out_index_name="Overall Index",
    out_index_reverse=None,
    post_min_max=None,
    post_reclass=None,
    post_num_classes=5,
    post_custom_classes=None
)